In [ ]:
# import internal modules
import file_path_management as fpath
import public_library as plib
import extract_info
import parameters as params

In [ ]:
# import packages
import csv
import pandas as pd
import PyPDF2
import requests
import time
import os
import random
from requests.auth import HTTPProxyAuth
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
import os
import re
from lxml import etree

In [ ]:
# extract text from given .pdf file
def pdf2text(pdf_path, text_path, page_start, page_end):
    try:
        # creating a pdf reader object
        reader = PyPDF2.PdfReader(pdf_path)
        
        # printing number of pages in pdf file
        page_max = len(reader.pages)
        
        # getting a specific page from the pdf file
        text = ""
        
        for i in range(page_start, page_end + 1):
            # print(i)
            page = reader.pages[i]
            text = text + "".join(page.extract_text().splitlines())

        with open(text_path, "w") as f:
            f.write(text)
        f.close()

        return True
    except:
        return False
# --------------------start of test code--------------------
# ind = 1
# pdf_file_name = str(index) + ".pdf"
# pdf_folder = fpath.litera_pdf_folder
# pdf_path = os.path.join(pdf_folder, pdf_file_name)

# text_folder_path = fpath.litera_text_folder
# text_file_name = pdf_file_name.split(".pdf")[0] + ".txt"
# text_path = os.path.join(text_folder_path, text_file_name)

# start_page = 0
# end_page = 1
# if not pdf2text(pdf_path, text_path, start_page, end_page):
#     print("Error: pdf2text() failed")
# ---------------------end of test code---------------------

In [ ]:
# download pdf to specified folder given pdf_url and ind
def download_pdf(pdf_url, pdf_folder):  
    url = pdf_url
    if url != url:
        raise Exception("pdf_url is np.nan")
    else:
        source = url.split("://")[1].split("/")[0]
    
    # pdf_path = os.path.join(pdf_folder, file_name)
      
    # get the pdf content
    # options = webdriver.ChromeOptions()
    # options.add_experimental_option('prefs', {
    # "download.default_directory": pdf_folder, #Change default directory for downloads
    # "download.prompt_for_download": False, #To auto download the file
    # "download.directory_upgrade": True,
    # "download.open_pdf_in_system_reader": False,
    # "plugins.always_open_pdf_externally": True #It will not show PDF directly in chrome
    # })
    # driver = webdriver.Chrome(options=options)
    # # response = driver.get(pdf_url)
    # driver.navigate().to(url)
    # time.sleep(5)
    if source == "www.sciencedirect.com":
        os.environ['WDM_LOG'] = '0'
        options = Options()
        options.add_argument('--headless')
        driver = webdriver.Chrome()
        driver.get(url)
        element = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, "//li[@class='ViewPDF']/a")))
        element.click()
        time.sleep(10)
    else: 
        options = webdriver.ChromeOptions()
        options.add_experimental_option('prefs', {
        "download.default_directory": pdf_folder, #Change default directory for downloads
        "download.prompt_for_download": False, #To auto download the file
        "download.directory_upgrade": True,
        "plugins.always_open_pdf_externally": True #It will not show PDF directly in chrome
        })
        driver = webdriver.Chrome(options=options)
        response = driver.get(url)
        time.sleep(10)
    driver.close()

    # try:
    #     with open(pdf_path, 'wb') as pdf_object:
    #         pdf_object.write(response.content)
    #         driver.close()
    #         return True
    # except:
    #     print(f'Failed downloading PDF:', url)
    #     driver.close()
    #     # print(f'HTTP response status code: {response.status_code}')
    #     return False
# --------------------start of test code--------------------
# # pdf_url = 'https://www.sciencedirect.com/science/article/pii/S0896627320300052/pdfft?md5=3f0648c6385e6fae3a5a73b053903014&pid=1-s2.0-S0896627320300052-main.pdf'
# # pdf_url = "https://www.sciencedirect.com/science/article/pii/S1053811909013299?via%3Dihub"
# # pdf_url = "https://linkinghub.elsevier.com/retrieve/pii/S1053811909013299"
# # pdf_url = "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1181753/pdf/jphysiol00456-0559.pdf"
# pdf_url = "https://www.sciencedirect.com/science/article/pii/S0896627320300052?via%3Dihub"

# # wiley.com
# # pdf_url = "https://onlinelibrary.wiley.com/doi/pdf/10.1002/cne.903130106"
# # pdf_url = "https://onlinelibrary.wiley.com/doi/pdfdirect/10.1002/cne.903130106"
# # pdf_url = "https://onlinelibrary.wiley.com/doi/pdf/10.1002/cne.902770204"
# # pdf_url = "https://onlinelibrary.wiley.com/doi/pdfdirect/10.1002/cne.902770204"
# # pdf_url = "https://onlinelibrary.wiley.com/doi/pdf/10.1002/cne.902970309"
# # pdf_url = "https://onlinelibrary.wiley.com/doi/pdfdirect/10.1002/cne.902970309"

# # physiology.org
# # pdf_url = "https://journals.physiology.org/doi/pdf/10.1152/jn.2001.85.1.219"

# # springer.com
# # pdf_url = "https://link.springer.com/content/pdf/10.1007/BF00236173.pdf"

# # cell.com
# # pdf_url = "https://www.cell.com/neuron/pdf/S0896-6273(09)00170-6.pdf"

# # jneurosci.org
# # pdf_url = "https://www.jneurosci.org/content/jneuro/22/18/8117.full.pdf"
# # pdf_url, code = plib.get_final_redirected_url(pdf_url)
# # print(pdf_url)
# ind = 10
# file_name = str(ind) + ".pdf"
# pdf_folder = fpath.litera_pdf_folder
# download_pdf(pdf_url, pdf_folder)
# # rename_pdf(file_name, pdf_folder, time_to_wait=60)
# ---------------------end of test code---------------------

In [ ]:
# download pdf
def download_pdf(pdf_url, pdf_source, pdf_folder):
    for website in params.website_pdfs:
        func = None
        if website in pdf_source:
            # Get the function name by replacing "." with "_" and use globals() to call it
            func_name = "download_from_" + website.replace(".", "_")
            # print(func_name)
            func = globals().get(func_name)
            # print(func)
            break
    if func != None:
        # print(func)
        func(pdf_url, pdf_folder)
    else:
        print("The given url is not from a supported website: ", pdf_url)
        raise Exception("Function does not exist for website:", pdf_url)

# rename downloaded pdf
def rename_pdf(ind, pdf_folder, time_to_wait=60):
    newname = str(ind) + ".pdf"
    time_counter = 0
    filename = max([f for f in os.listdir(pdf_folder)], key=lambda xa :   os.path.getctime(os.path.join(pdf_folder,xa)))
    while '.part' in filename:
        time.sleep(1)
        time_counter += 1
        if time_counter > time_to_wait:
            raise Exception('Waited too long for file to download')
    filename = max([f for f in os.listdir(pdf_folder)], key=lambda xa :   os.path.getctime(os.path.join(pdf_folder,xa)))
    os.rename(os.path.join(pdf_folder, filename), os.path.join(pdf_folder, newname))

# download and rename pdf
def download_and_rename_pdf(pdf_url, ind, pdf_folder):  
    if pdf_url != pdf_url:
        raise Exception("pdf_url is np.nan")
    else:
        pdf_source = pdf_url.split("://")[1].split("/")[0]
    download_pdf(pdf_url, pdf_source, pdf_folder)
    rename_pdf(ind, pdf_folder, time_to_wait=60)
    print(f"Downloaded {ind}.pdf")
# --------------------start of test code--------------------
# pdf_url = "https://journals.physiology.org/doi/pdf/10.1152/jn.2001.85.1.219"
# ind = 10
# pdf_folder = "/home/hou/myProjects/litera_pdfs"
# download_and_rename_pdf(pdf_url, ind, pdf_folder)
# ---------------------end of test code---------------------
# https://journals.physiology.org/doi/10.1152/jn.2001.85.1.219

In [ ]:
# full_text_url = 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5856726/'
# doi = '10.1007/s12264-018-0210-1'
# pmid = '29430586'
# pmcid = 'PMC5856726'
# if full_text_url == full_text_url and full_text_url.split("://")[1].split("/")[0] == 'www.ncbi.nlm.nih.gov':
#     if doi == doi:
#         url = "https://doi.org/" + doi
#         url, status_code = plib.get_final_redirected_url(url)
#         info = extract_info.extract_info_from_webpage(url, params.websites)
#         full_text_url = url
#         full_text_source = url.split("://")[1].split("/")[0]
#         pdf_url = info["pdf_link"]
#         pdf_source = pdf_url.split("://")[1].split("/")[0]
#     else:
#         pdf_url = np.nan
#         pdf_source = np.nan
# else:  
#     print("PDF_URL not found for: ", doi, pmid, pmcid, full_text_url)
#     pdf_url = np.nan
#     pdf_source = np.nan
# print(doi)
# print(pmid)
# print(pmcid)
# print(full_text_url)
# print(full_text_source)
# print(pdf_url)
# print(pdf_source)

In [ ]:
# # "//button[@class='dropdown-trigger btn btn--light btn--cta_roundedColored']"
# ['www.ahajournals.org', 'psychiatryonline.org']
# 'www.ahajournals.org'
# "https://www.ahajournals.org/doi/reader/10.1161/01.STR.0000087786.38997.9E"
# # 'neuro.psychiatryonline.org'
# # 'ajp.psychiatryonline.org'
# "https://neuro.psychiatryonline.org/doi/reader/10.1176/jnp.16.2.127"
# "https://ajp.psychiatryonline.org/doi/reader/10.1176/appi.ajp.158.9.1411"

In [ ]:
# # "//a[@class='navbar-download btn btn--cta_roundedColored']"
# ['wiley.com', 'www.science.org', 'physiology.org', 'tandfonline.com', 'sagepub.com', 'acs.org']
# # 'anatomypubs.onlinelibrary.wiley.com'
# # 'nyaspubs.onlinelibrary.wiley.com'
# # 'physoc.onlinelibrary.wiley.com'
# # 'onlinelibrary.wiley.com'
# # 'analyticalsciencejournals.onlinelibrary.wiley.com'
# # 'movementdisorders.onlinelibrary.wiley.com'
# "https://physoc.onlinelibrary.wiley.com/doi/epdf/10.1113/JP280844"
# "https://nyaspubs.onlinelibrary.wiley.com/doi/epdf/10.1196/annals.1284.033"
# "https://anatomypubs.onlinelibrary.wiley.com/doi/epdf/10.1002/ar.22454"
# "https://onlinelibrary.wiley.com/doi/epdf/10.1111/ejn.14426"
# "https://analyticalsciencejournals.onlinelibrary.wiley.com/doi/epdf/10.1002/jemt.10404"
# "https://movementdisorders.onlinelibrary.wiley.com/doi/epdf/10.1002/mds.870060404"
# # www.science.org
# "https://www.science.org/doi/reader/10.1126/science.282.5391.1117"
# # journals.physiology.org
# "https://journals.physiology.org/doi/epdf/10.1152/jn.2001.85.1.219"
# # www.tandfonline.com
# "https://www.tandfonline.com/doi/epdf/10.3109/08990229109144753?needAccess=true&role=button"
# # journals.sagepub.com
# "https://journals.sagepub.com/doi/reader/10.1177/107385840100700408"
# # pubs.acs.org
# "https://pubs.acs.org/doi/epdf/10.1021/jm030384e"

In [ ]:
# # .pdf
# ['aspetjournals.org', 'citeseerx.ist.psu.edu', 'www.nature.com', 'karger.com', 'ahuman.org', 'ahuman.org', 'www.researchsquare.com',
#  'link.springer.com', 'www.ijpp.com', 'www.ijpp.com', 'www.cell.com', 'www.bu.edu', 'www.ncbi.nlm.nih.gov', 
#  'www.thieme-connect.de', 'deepblue.lib.umich.edu', 'bpb-us-e1.wpmucdn.com', 'www.researchgate.net', 'ieeexplore.ieee.org',
#  'zsp.com.pk', 'journals.biologists.com', 'journals.aps.org', 'academic.oup.com', 'www.biorxiv.org', 'enpubs.faculty.ucdavis.edu',
#  'n.neurology.org', 'ruor.uottawa.ca', 'www.jstage.jst.go.jp', 'synapse.koreamed.org', 'www.jneurosci.org', 'pubs.asahq.org',
#  'biomedcentral.com', 'direct.mit.edu', 'jnm.snmjournals.org']

# # pharmrev.aspetjournals.org
# # jpet.aspetjournals.org
# "https://pharmrev.aspetjournals.org/content/pharmrev/24/1/31.full.pdf"
# "https://jpet.aspetjournals.org/content/jpet/325/2/617.full.pdf"
# "https://jpet.aspetjournals.org/content/jpet/321/1/116.full.pdf"
# # citeseerx.ist.psu.edu
# "https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=086111ccf8db5585f16a54ba754ea75ebac97d6c"
# # www.nature.com
# "https://www.nature.com/articles/s41586-020-2914-4.pdf"
# # karger.com
# "https://karger.com/bbe/article-pdf/6/1-6/409/2259371/000315942.pdf"
# # ahuman.org
# "http://ahuman.org/svn/ahengine/research/articles/Biological/2002-Pathways-for-emotions-and-memory.PDF"
# # www.researchsquare.com
# "https://www.researchsquare.com/article/rs-2921649/v1.pdf?c=1687584387000"
# # link.springer.com
# "https://link.springer.com/content/pdf/10.1007/BF00236173.pdf?pdf=button"
# # www.ijpp.com
# "https://www.ijpp.com/IJPP%20archives/1981_25_3/201-208.pdf"
# # www.cell.com
# "https://www.cell.com/neuron/pdf/S0896-6273(11)00557-5.pdf"
# # www.bu.edu
# "https://www.bu.edu/neural/Final/Publications/2002/Thalamus%20%26%20Related%20Systems%2C%20Volume%202%2C%20Issue%201%2C%20December%202002%2C%20Pages%2021-32.pdf"
# # www.ncbi.nlm.nih.gov
# "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6577493/pdf/jneuro_14_5_2485.pdf"
# # www.thieme-connect.de
# "https://www.thieme-connect.de/products/ejournals/pdf/10.1055/s-2007-973495.pdf"
# # deepblue.lib.umich.edu
# "https://deepblue.lib.umich.edu/bitstream/handle/2027.42/74092/annals.1300.030.pdf?sequence=1"
# # bpb-us-e1.wpmucdn.com
# "https://bpb-us-e1.wpmucdn.com/sites.northwestern.edu/dist/7/2577/files/2018/08/23-Mesulam-2oebbn3.pdf"
# # www.researchgate.net
# "https://www.researchgate.net/profile/James-Augustine-2/publication/20294840_Augustine_JR_The_insular_lobe_in_primates_including_humans_Neurol_Res_7_2-10/links/60651633299bf1252e1cf66d/Augustine-JR-The-insular-lobe-in-primates-including-humans-Neurol-Res-7-2-10.pdf"
# # ieeexplore.ieee.org
# "https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=5333751"
# # zsp.com.pk
# "http://zsp.com.pk/pdf48/575-581%20(38)%20QPJZ-0063-2015%2017-12-15%20Longitudinal%20Metabolic%20Changes%20in%20the%20Thalamus%20o_.pdf"
# # journals.biologists.com
# "https://journals.biologists.com/dev/article-pdf/117/3/1031/3050703/develop_117_3_1031.pdf"
# # journals.aps.org
# "https://journals.aps.org/pre/pdf/10.1103/PhysRevE.106.054304"
# # academic.oup.com
# "https://academic.oup.com/cercor/article-pdf/10/3/220/9751036/100220.pdf"
# # www.biorxiv.org
# "https://www.biorxiv.org/content/10.1101/2022.02.03.479036v2.full.pdf"
# # enpubs.faculty.ucdavis.edu
# "https://enpubs.faculty.ucdavis.edu/wp-content/uploads/sites/209/2015/03/Disbrow-et-al_2002-Thalamocoretical-connections-of-the-parietal-ventral-area-and-the-second.pdf"
# # n.neurology.org
# "https://n.neurology.org/content/neurology/64/6/1014.full-text.pdf"
# # ruor.uottawa.ca
# "https://ruor.uottawa.ca/bitstream/10393/8164/1/NK25433.PDF"
# # www.jstage.jst.go.jp
# "https://www.jstage.jst.go.jp/article/pjab1945/43/8/43_8_822/_pdf/-char/en"
# # synapse.koreamed.org
# "https://synapse.koreamed.org/upload/synapsedata/pdfdata/0069ymj/ymj-55-709.pdf"
# # www.jneurosci.org
# "https://www.jneurosci.org/content/jneuro/11/8/2383.full.pdf"
# # pubs.asahq.org
# "https://pubs.asahq.org/anesthesiology/article-pdf/98/5/1101/407475/0000542-200305000-00012.pdf"
# # biomedcentral.com
# "https://biomedical-engineering-online.biomedcentral.com/counter/pdf/10.1186/1475-925X-3-13.pdf"
# # direct.mit.edu
# "https://direct.mit.edu/jocn/article-pdf/19/1/13/1936104/jocn.2007.19.1.13.pdf"
# # jnm.snmjournals.org
# "https://jnm.snmjournals.org/content/jnumed/39/2/281.full.pdf"

In [ ]:
# # request
# ['iovs.arvojournals.org', 'www.imrpress.com', 'www.hifo.uzh.ch', 'ujms.net', 'www.annualreviews.org', 'thejns.org']
# # iovs.arvojournals.org
# "https://iovs.arvojournals.org//arvo/content_public/journal/iovs/934840/i1552-5783-57-1-1.pdf"
# # www.imrpress.com
# "https://www.imrpress.com/journal/JIN/20/1/10.31083/j.jin.2021.01.334/pdf"
# # www.hifo.uzh.ch
# "https://www.hifo.uzh.ch/dam/jcr:00000000-2999-c151-ffff-ffffd9509b89/paper_CorticalArea8.pdf"
# # ujms.net
# "https://ujms.net/index.php/ujms/article/download/6812/12603"
# # www.annualreviews.org
# "https://www.annualreviews.org/doi/pdf/10.1146/annurev.ne.11.030188.001345"
# # thejns.org
# "https://thejns.org/downloadpdf/journals/j-neurosurg/86/1/article-p77.xml"

In [ ]:
# # www.microbiologyresearch.org
# "://www.microbiologyresearch.org/"
# "https://www.microbiologyresearch.org/content/journal/jgv/10.1099/vir.0.79883-0"
# # europepmc.org
# "://europepmc.org/"
# "https://europepmc.org/article/med/8784824"
# # jamanetwork.com
# "://jamanetwork.com/"
# "https://jamanetwork.com/journals/jamaneurology/article-abstract/565945"
# # papers.ssrn.com
# "://papers.ssrn.com/"
# "https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3689615"
# # royalsocietypublishing.org
# "://royalsocietypublishing.org/"
# "https://royalsocietypublishing.org/doi/10.1098/rspb.1953.0054"
# # www.ingentaconnect.com
# "://www.ingentaconnect.com/"
# "https://www.ingentaconnect.com/content/aalas/cm/2000/00000050/00000002/art00006"
# # linkinghub.elsevier.com
# "://linkinghub.elsevier.com/"
# "1296,10.1016/j.neures.2006.02.006"
# "https://linkinghub.elsevier.com/retrieve/pii/S0168010206000423"
# # journals.lww.com
# "://journals.lww.com/"
# "https://journals.lww.com/neuroreport/abstract/1994/10000/further_evidence_for_two_types_of_corticopulvinar.6.aspx"

In [ ]:
def download_by_request(url, ind, pdf_folder):
    try:
        file_name = str(ind) + ".pdf"
        response = requests.get(url, headers=plib.headers)
        
        # download the .pdf file to the pdf_file_path folder
        # write content in pdf file
        pdf_path = os.path.join(pdf_folder, file_name)
        
        if response.status_code == 200:
            with open(pdf_path, 'wb') as pdf_object:
                pdf_object.write(response.content)
            print(f'Successfully downloaded PDF:', ind)
        else:
            print(f'Failed downloading PDF:' + 'pdf_url')
            print(f'HTTP response status code: {response.status_code}')
    except:
        print(f'Failed downloading PDF:' + 'pdf_url')
# --------------------start of test code--------------------
pdf_url = "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6577493/pdf/jneuro_14_5_2485.pdf"
# pdf_url = "https://pharmrev.aspetjournals.org/content/pharmrev/24/1/31.full.pdf"
# pdf_url = "https://jnm.snmjournals.org/content/jnumed/39/2/281.full.pdf"
ind = 10
pdf_folder = fpath.pdf_folder
download_by_request(pdf_url, ind, pdf_folder)
# ---------------------end of test code---------------------

In [ ]:
# def download_from_ELSEVIER(doi, file_to_save_to):
#     url = 'http://api.elsevier.com/content/article/doi:'+doi+'?view=FULL'
#     headers = {
#         'X-ELS-APIKEY': "63f58b8b10cbc1bc923011c01c6301bb",
#         'Accept': 'application/pdf'
#     }
#     r = requests.get(url, stream=True, headers=headers)
#     if r.status_code == 200:
#         for chunk in r.iter_content(chunk_size=1024*1024):
#             file_to_save_to.write(chunk)
#         return True

# # doi_list = pd.read_excel('list.xls')
# # doi_list.columns = ['DOIs']
# count = 0
# # for doi in doi_list['DOIs']:
# doi = "10.1016/0006-8993(95)01338-5"
# # pdf = "10"
# ind = 10
# file_name = str(ind) + ".pdf"
# # if not os.path.exists(f'path/{pdf}.pdf'):
# with open(file_name, 'wb') as pdf_object:
#     get_pdf(doi, pdf_object)
# pdf_object.close